# Mathematical Optimisation - Project
This project aims at ricreating the results reported in:

[A fix-and-optimize heuristic for the Unrelated Parallel Machine Scheduling Problem
](https://www.sciencedirect.com/science/article/pii/S0305054823003684)

In [1]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output

r.seed(3000)
  
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

## Creation of the matrices

- P stores the execution times
- S stores the setup times

These matrices are converted to dictionaries to handle the indexes not starting from 0

In [2]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 10,
                      min_setup_time = 1,max_setup_time = 3):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 10
M_cardinality = 2

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality)
print(f'P: {P_dict}')
print(f'S: {S_dict}')

P: {(1, 1): 3, (1, 2): 5, (1, 3): 2, (1, 4): 5, (1, 5): 5, (1, 6): 3, (1, 7): 10, (1, 8): 8, (1, 9): 5, (1, 10): 4, (2, 1): 3, (2, 2): 9, (2, 3): 4, (2, 4): 4, (2, 5): 9, (2, 6): 6, (2, 7): 10, (2, 8): 10, (2, 9): 7, (2, 10): 7}
S: {(1, 1, 1): 2, (1, 1, 2): 3, (1, 1, 3): 3, (1, 1, 4): 2, (1, 1, 5): 2, (1, 1, 6): 1, (1, 1, 7): 3, (1, 1, 8): 2, (1, 1, 9): 2, (1, 1, 10): 1, (1, 2, 1): 3, (1, 2, 2): 2, (1, 2, 3): 3, (1, 2, 4): 2, (1, 2, 5): 1, (1, 2, 6): 1, (1, 2, 7): 1, (1, 2, 8): 3, (1, 2, 9): 2, (1, 2, 10): 2, (1, 3, 1): 1, (1, 3, 2): 1, (1, 3, 3): 3, (1, 3, 4): 3, (1, 3, 5): 3, (1, 3, 6): 1, (1, 3, 7): 3, (1, 3, 8): 3, (1, 3, 9): 2, (1, 3, 10): 1, (1, 4, 1): 2, (1, 4, 2): 2, (1, 4, 3): 3, (1, 4, 4): 3, (1, 4, 5): 1, (1, 4, 6): 3, (1, 4, 7): 2, (1, 4, 8): 3, (1, 4, 9): 1, (1, 4, 10): 1, (1, 5, 1): 1, (1, 5, 2): 1, (1, 5, 3): 2, (1, 5, 4): 3, (1, 5, 5): 1, (1, 5, 6): 2, (1, 5, 7): 3, (1, 5, 8): 3, (1, 5, 9): 3, (1, 5, 10): 1, (1, 6, 1): 1, (1, 6, 2): 2, (1, 6, 3): 1, (1, 6, 4): 1, (1, 6,

In [3]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

# Exact Solution

The problem is solved exactly using the Solver class.

Then, the results are printed. Note that job 0 is a dummy job used to represent the beginning and the end of activities for each machine.

In [4]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)

for i in s.M:
    for j in s.N:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

print(f'The makespan has been minimized to {maximum_makespan}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Job 1 scheduled in sequence of job 0 in machine 1
Job 2 scheduled in sequence of job 6 in machine 1
Job 5 scheduled in sequence of job 10 in machine 1
Job 6 scheduled in sequence of job 1 in machine 1
Job 9 scheduled in sequence of job 5 in machine 1
Job 10 scheduled in sequence of job 2 in machine 1
Job 3 scheduled in sequence of job 7 in machine 2
Job 4 scheduled in sequence of job 0 in machine 2
Job 7 scheduled in sequence of job 4 in machine 2
Job 8 scheduled in sequence of job 3 in machine 2
Job 1 completed at 30.0s
Job 2 completed at 22.0s
Job 3 completed at 15.0s
Job 4 completed at 31.0s
Job 5 completed at 11.0s
Job 6 completed at 26.0s
Job 7 completed at 26.0s
Job 8 completed at 10.0s
Job 9 completed at 5.0s
Job 10 completed at 16.0s
The makespan has been minimized to 31.0s
Job 1 has been assign

# Mathematical Programming Algorithm (MPA)

The paper cites a decomposition algorithm (Branch-and-check) based on the previous formulation.

It is an exact formulation based on a master algorithm and a sequence algorithm:
- The master algorithm is responsible of assigning jobs to machines
- The sequence problem is responsible of finding the optimal sequence of jobs for each machine

In [5]:
s = MPA(execution_times = P_dict, setup_times = S_dict)
decision_variables,maximum_makespan = s.solve(options=options)
for i in s.M:
    for j in s.N:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
1
C_max_h  {1: {1: 24.0, 2: 25.0}}
self.M  range(1, 3)
C_max_h  {1: {1: 24.0, 2: 25.0}}
Y  {(1, 0): <gurobi.Var *Awaiting Model Update*>, (1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>, (1, 4): <gurobi.Var *Awaiting Model Update*>, (1, 5): <gurobi.Var *Awaiting Model Update*>, (1, 6): <gurobi.Var *Awaiting Model Update*>, (1, 7): <gurobi.Var *Awaitin

TypeError: 'float' object is not iterable

# Fix-and-Optimize approach

In [ ]:
greedy = Greedy(execution_times=P_dict, setup_times=S_dict)
solution = greedy.solve()

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for j in s.N:
    print(f'Job {j} completed at {completion_times[j]}s')

In [ ]:
m = FixOpt(initial_solution=solution, setup_times=S_dict,execution_times=P_dict, N=N, M=M, N0=N0, 
           subproblem_size_adjust_rate=0.1, t_max = 30, subproblem_runtime_limit=10, subproblem_size=5)

solution, makespan = m.solve()
clear_output(wait=True)
for i in M:
    for j in N0:
        for k in N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled after job {k} in machine {i}')
print(f'The makespan has been minimized to {makespan}s')